In [1]:
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import accuracy_score
import os
import glob
from keras.models import Sequential
import pandas as pd
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import keras

In [2]:
alpha_dic = {2.0:0.25,1.9:0.35,1.5:0.2,1.3:0.15,1.0:0.1,0.9:0.1,'mixture':0.25,'multiple':0.25,0.0:0.1}

In [3]:
folder_path = "/home/xueqiongyuan/new_noise_code/codes/mnist_c"
# 使用 os.walk() 函数遍历文件夹及其子文件夹
# 获取文件夹中所有的子文件夹名称
subfolder_names = [name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))]

print(subfolder_names)

['motion_blur', 'glass_blur', 'scale', 'identity', 'canny_edges', 'rotate', 'shear', 'dotted_line', 'brightness', 'stripe', 'translate', 'shot_noise', 'impulse_noise', 'zigzag', 'spatter', 'fog']


In [4]:
alphas = [0.0,2.0,'multiple',1.0] #5.5是multiple
column_names = ['noise type']+subfolder_names
df = pd.DataFrame(columns=column_names)
df['noise type'] = alphas
width = 100 #3,10,100
depth = 3 #3,4,5
b = 64
for package in subfolder_names:
    acc_final_list = []

    x_adv = np.load(f"/home/xueqiongyuan/new_noise_code/codes/mnist_c/{package}/test_images.npy")
    y_te = np.load(f"/home/xueqiongyuan/new_noise_code/codes/mnist_c/{package}/test_labels.npy")
    
    x_adv = x_adv/255
    
    x_adv = (x_adv - 0.1307) / 0.3081
    x_adv = x_adv.reshape(10000,784)
    for alpha in alphas:
        print(f'--alpha = {alpha} --')
        acc_list = []
        for num in range(5):
            if depth != 3:
                lr = 0.0005
                iters = 50000
            else:
                lr = 0.001
                iters = 10000
            model = Sequential()
            for j in range(depth-2):
                model.add(Dense(width, activation='relu',input_shape=(1,784)))
            model.add(Dense(10, activation='softmax'))
            model.compile(loss=keras.losses.categorical_crossentropy,
                                optimizer=Adam(lr=lr),
                                metrics=['accuracy'])
            print(f'-- repeat time {num} --')
            model.load_weights('/home/xueqiongyuan/new_noise_code/models/mnist_FCN_width{}_depth{}_b{}_alpha{}_sigma{}_num{}_lr{}_iter{}.h5'.format(width, depth, b, alpha, alpha_dic[alpha], num, lr, iters))
            y_pred = model.predict(x_adv)
            encoder = LabelEncoder()
            y_test = encoder.fit_transform(y_te)
            y_pred_label = np.argmax(y_pred, axis=1).flatten()
            y_pred_label = encoder.fit_transform(y_pred_label)
            acc = accuracy_score(y_test, y_pred_label)
            acc_list.append(acc)
        acc_final_list.append(np.mean(acc_list))
    df[package]=acc_final_list
   
df.to_csv('mnist_c_results_w{}_d{}.csv'.format(width,depth), index=False) 

--alpha = 0.0 --
-- repeat time 0 --
-- repeat time 1 --
-- repeat time 2 --
-- repeat time 3 --
-- repeat time 4 --
--alpha = 2.0 --
-- repeat time 0 --
-- repeat time 1 --
-- repeat time 2 --
-- repeat time 3 --
-- repeat time 4 --
--alpha = multiple --
-- repeat time 0 --
-- repeat time 1 --
-- repeat time 2 --
-- repeat time 3 --
-- repeat time 4 --
--alpha = 1.0 --
-- repeat time 0 --
-- repeat time 1 --
-- repeat time 2 --
-- repeat time 3 --
-- repeat time 4 --
--alpha = 0.0 --
-- repeat time 0 --
-- repeat time 1 --
-- repeat time 2 --
-- repeat time 3 --
-- repeat time 4 --
--alpha = 2.0 --
-- repeat time 0 --
-- repeat time 1 --
-- repeat time 2 --
-- repeat time 3 --
-- repeat time 4 --
--alpha = multiple --
-- repeat time 0 --
-- repeat time 1 --
-- repeat time 2 --
-- repeat time 3 --
-- repeat time 4 --
--alpha = 1.0 --
-- repeat time 0 --
-- repeat time 1 --
-- repeat time 2 --
-- repeat time 3 --
-- repeat time 4 --
--alpha = 0.0 --
-- repeat time 0 --
-- repeat time 1 

In [5]:
df

,noise type,motion_blur,glass_blur,scale,identity,canny_edges,rotate,shear,dotted_line,brightness,stripe,translate,shot_noise,impulse_noise,zigzag,spatter,fog
0,0,0.81066,0.91596,0.68228,0.97788,0.61930,0.83988,0.92756,0.94172,0.25984,0.16732,0.28956,0.96200,0.90184,0.77614,0.94998,0.30832
1,2,0.83722,0.93348,0.75040,0.98014,0.63390,0.85402,0.93878,0.96114,0.45132,0.23578,0.30926,0.96964,0.94522,0.87022,0.96314,0.44730
2,multiple,0.84610,0.94622,0.76436,0.97992,0.63496,0.86496,0.93998,0.97034,0.86582,0.50620,0.29072,0.97340,0.96242,0.91902,0.96844,0.71652
3,1,0.85208,0.94578,0.72250,0.97958,0.65404,0.86300,0.93770,0.96550,0.56250,0.44430,0.30560,0.97094,0.95006,0.88980,0.96486,0.52882
